In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='8um'

In [4]:
design.get_chip_z('Q_chip')

'8um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="110um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [13]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [56]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [57]:
hfss.start()

INFO 03:38PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:38PM [load_ansys_project]: 	Opened Ansys App
INFO 03:38PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:38PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 03:38PM [connect_design]: 	Opened active design
	Design:    flip7um_V6 [Solution type: Eigenmode]
INFO 03:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:38PM [connect]: 	Connected to project "Flipchip Bump" and design "flip7um_V6" 😀 



True

In [58]:
hfss.activate_ansys_design("flip8um_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:38PM 54s WARNING [activate_ansys_design]: The design_name=flip8um_V6 was not in active project.  Designs in active project are: 
['flip7um_V6', 'flip6um_V6'].  A new design will be added to the project.  
INFO 03:38PM [connect_design]: 	Opened active design
	Design:    flip8um_V6 [Solution type: Eigenmode]
WARNING 03:38PM [connect_setup]: 	No design setup detected.
WARNING 03:38PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [59]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [14]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [15]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.67',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.67',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [17]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 03:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpb56cruxk.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 1, False
INFO 03:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0gigyvlf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 1, False
INFO 03:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp50oy691c.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 2, False
INFO 03:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpq0bcuiah.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 3, False
INFO 03:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp14jy0x2n.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 4, False
INFO 03:03

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,342.510,-108.280000,-93.954,-77.009000
pad_bot_Q0,-108.280,122.100000,-12.346,-0.056553
pad_top_Q0,-93.954,-12.346000,121.990,-14.263000
quad_coupler_c0,-77.009,-0.056553,-14.263,91.492000


In [18]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 6.76
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.252159 [GHz]
EC 282.841583 [MHz]
EJ 13.616300 [GHz]
alpha -325.954495 [MHz]
dispersion 15.388869 [KHz]
Lq 11.995161 [nH]
Cq 68.484372 [fF]
T1 71.129682 [us]

**Coupling Properties**

tCqbus1 -7.107176 [fF]
gbus1_in_MHz -99.201940 [MHz]
χ_bus1 -2.365876 [MHz]
1/T1bus1 2237.532050 [Hz]
T1bus1 71.129682 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.626034,327.589793,13.6163,-382.904591,58.656786,[-119.05816894171298],[-6.383901363898839],6.383901,119.058169
2,5.411677,301.473654,13.6163,-349.492795,27.870314,[-115.8084641799818],[-4.160244222345131],4.160244,115.808464
3,5.319781,290.65722,13.6163,-335.798118,19.882557,[-105.7063343845728],[-2.9867266220485242],2.986727,105.706334
4,5.278519,285.873762,13.6163,-329.768319,17.018613,[-101.58414975479306],[-2.5846763890494393],2.584676,101.584150
5,5.271657,285.082732,13.6163,-328.772747,16.580118,[-100.23021788783386],[-2.489432036104353],2.489432,100.230218
6,5.259545,283.68935,13.6163,-327.020146,15.830747,[-99.5345964803324],[-2.4091839405351476],2.409184,99.534596
7,5.252159,282.841583,13.6163,-325.954495,15.388869,[-99.20193986477142],[-2.3658763009422805],2.365876,99.201940


In [19]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
